In [2]:
%reload_ext autoreload
%autoreload

In [5]:
# export pod.client

In [6]:
import hashlib
import json
import os
import requests
import hashlib
import json
import os
import requests

class PodClient:
    def __init__(self, run_item=None):
        if 'POD_FULL_ADDRESS' in os.environ:
            self.pod_full_address = os.environ['POD_FULL_ADDRESS']
            print(f'Running with POD_FULL_ADDRESS (from env variable) : {self.pod_full_address}')
        else:
            self.pod_full_address = 'https://0.0.0.0:3030'
            print(f'Running with POD_FULL_ADDRESS (from default) as {self.pod_full_address}')

        if run_item == None:
            # TODO: parse run variables from env variable once exact 
            # name is known (likely to be RUN_ITEM)
            
            # self.run_item = json.loads(self.get_run_item())
            print('RUN_VARIABLES are taken from environment variable')
        else:
            self.run_item = run_item
            print('RUN_VARIABLES are injected/side-loaded')
        self.uid = self.run_item['uid']
        print('RUN_VARIABLES are ', self.run_item)

        self.client_payload = json.loads(self.get_client_payload())
        self.database_key = self.client_payload['databaseKey']
        self.owner_key = self.client_payload['ownerKey']
        
        
    def get_run_variable(self, variable):
        return self.run_item[variable]

#     def get_run_item(self):
#         json_object = {
#             'uid': 'xxx'
#         }
#         if False:
#             return json.dumps(json_object)
#         else:
#             settings_file = open('local_settings.json', 'r')
#             return settings_file.read()

    def get_client_payload(self):
        # TODO: parse env variable once exact 
        # name is known (likely to be CLIENT_PAYLOAD)
        json_object = {
            'databaseKey': 'database_key',
            'ownerKey': 'owner_key',
        }
        return json.dumps(json_object)

    def create_item(self, item, item_type=None):
        if item_type != None:
            item['_type'] = item_type

        wrapped_item = {
            'databaseKey': self.database_key,
            'payload': item,
        }
        result = requests.post(f'{self.pod_full_address}/v2/{self.owner_key}/create_item',
                      json.dumps(wrapped_item),
                      verify=False)

        if result.ok:
            return result.json()
        else:
            # TODO: better exception handling
            raise Exception(result.status_code, result.text)

    def get_item(self, uid):
        wrapped_item = {
            'databaseKey': self.database_key,
            'payload': uid,
        }

        result = requests.post(f'{self.pod_full_address}/v2/{self.owner_key}/get_item',
                      json.dumps(wrapped_item),
                      verify=False)

        if result.ok:
            # TODO: assumes the pod returns an array of length 1
            result_json = result.json()
            if len(result_json) == 0:
                raise Exception(f'item {uid} does not exist')
            else:
                return result_json[0]
        else:
            # TODO: better exception handling
            raise Exception(result.status_code, result.text)

    def upload_file(self, file):
        hash = hashlib.sha256(file).hexdigest()

        # TODO: check if creation was succesful
        file_item = {
            'sha256': hash
        }
        self.create_item(file_item, item_type='File')

        result = requests.post(f'{self.pod_full_address}/v2/{self.owner_key}/upload_file/{self.database_key}/{hash}',
                               file,
                               verify=False)

        if result.ok:
            return hash
        else:
            raise Exception(result.status_code, result.text)

    def get_file(self, hash):
        wrapped_item = {
            'databaseKey': self.database_key,
            'payload': {
                'sha256': hash,
            }
        }

        result = requests.post(f'{self.pod_full_address}/v2/{self.owner_key}/get_file',
                               json.dumps(wrapped_item),
                               verify=False)

        if result.ok:
            return result.content
        else:
            raise Exception(result.status_code, result.text)

    def set_progress(self, new_progress):
        print('Progress of {} is now {}'.format(self.uid, new_progress))

IndentationError: expected an indented block (<ipython-input-6-16aa1692fdef>, line 29)

In [1]:
import shutil
from pathlib import Path

class DryRunPodClient(PodClient):
    def __init__(self, run_item=None):
        PodClient.__init__(self, run_item=run_item)
#         if Path('local/files').is_dir():
#             shutil.rmtree('local/files')
#         if Path('local/nodes').is_dir():
#             shutil.rmtree('local/nodes')
#         os.mkdir('local/files')
#         os.mkdir('local/nodes')

    def create_item(self, item, item_type=None):
#         if item_type != None:
#             item['_type'] = item_type

#         f = open('local/nodes/{}.json'.format(item['external_id']), "w")
#         f.write(json.dumps(item))
#         f.close()
        print("Create ITEM {}".format(item['external_id']))

    def upload_file(self, file):
        hash = hashlib.sha256(file).hexdigest()
#         f = open('local/files/{}.{}'.format(hash, 'jpg'), "wb")
#         f.write(file)
#         f.close()
        print("Uploaded FILE {}".format(hash))

NameError: name 'PodClient' is not defined